# Model Packaging Example

## Before Everything

### Snowflake-ML-Python Installation

- Please refer to our [landing page](https://docs.snowflake.com/en/developer-guide/snowpark-ml/index) to install `snowflake-ml-python`.

### Local Installation

- transformers>=4.31.0 (For GPT-2 and LLAMA 2 model inference example)
- tokenizers>=0.13.3 (For LLAMA 2 model inference example)
- tensorflow (For GPT-2 Example)
- xgboost==1.7.6 (For XGBoost GPU inference example)

### Additional Requirements

- SPCS compute pool with at least 1 GPU (For all GPU inference on SPCS examples below)

- Requested access to use LLama 2 model through HuggingFace (For LLAMA 2 model inference example)

- A HuggingFace token with read access (For LLAMA 2 model inference example)

- Download the News Category Dataset from https://www.kaggle.com/datasets/rmisra/news-category-dataset (For LLAMA 2 model inference example)

### Setup Notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Scale cell width with the browser window to accommodate .show() commands for wider tables.
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

### Start Snowpark Session

To avoid exposing credentials in Github, we use a small utility `SnowflakeLoginOptions`. It allows you to score your default credentials in `~/.snowsql/config` in the following format:
```
[connections]
accountname = <string>   # Account identifier to connect to Snowflake.
username = <string>      # User name in the account. Optional.
password = <string>      # User password. Optional.
dbname = <string>        # Default database. Optional.
schemaname = <string>    # Default schema. Optional.
warehousename = <string> # Default warehouse. Optional.
#rolename = <string>      # Default role. Optional.
#authenticator = <string> # Authenticator: 'snowflake', 'externalbrowser', etc
```
Please follow [this](https://docs.snowflake.com/en/user-guide/snowsql-start.html#configuring-default-connection-settings) for more details.

In [15]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

session = Session.builder.configs(SnowflakeLoginOptions()).create()

### Open/Create Model Registry

A model registry needs to be created before it can be used. The creation will create a new database in the current account so the active role needs to have permissions to create a database. After the first creation, the model registry can be opened without the need to create it again.

In [16]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY"
REGISTRY_SCHEMA_NAME = "PUBLIC"

In [ ]:
from snowflake.ml.registry import model_registry

model_registry.create_model_registry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)
registry = model_registry.ModelRegistry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)

## Use with scikit-learn model

### Train A Small Scikit-learn Model

The cell below trains a small model for demonstration purposes. The nature of the model does not matter, it is purely used to demonstrate the usage of the Model Packaging and Registry.

In [6]:
from sklearn import svm
from sklearn.datasets import load_digits
import numpy as np

digits = load_digits()
target_digit = 6
num_training_examples = 10
svc_gamma = 0.001
svc_C = 10.0

clf = svm.SVC(gamma=svc_gamma, C=svc_C, probability=True)


def one_vs_all(dataset, digit):
    return [x == digit for x in dataset]


# Train a classifier using num_training_examples and use the last 100 examples for test.
train_features = digits.data[:num_training_examples]
train_labels = one_vs_all(digits.target[:num_training_examples], target_digit)
clf.fit(train_features, train_labels)

test_features = digits.data[-100:]
test_labels = one_vs_all(digits.target[-100:], target_digit)
prediction = clf.predict(test_features)

In [ ]:
print(prediction[:10])

SVC has multiple method, for example, `predict_proba`.

In [ ]:
prediction_proba = clf.predict_proba(test_features)
print(prediction_proba[:10])

### Register Model

The call to `log_model` executes a few steps:
1. The given model object is serialized and uploaded to a stage.
1. An entry in the Model Registry is created for the model, referencing the model stage location.
1. Additional metadata is updated for the model as provided in the call.

For the serialization to work, the model object needs to be serializable in python.

Aso, you have to provide a sample input data so that we could infer the model signature for you, or you can specify the model signature manually.

In [ ]:
SVC_MODEL_NAME = "SIMPLE_SVC_MODEL"
SVC_MODEL_VERSION = "v1"

In [ ]:
# A name and model tags can be added to the model at registration time.
svc_model = registry.log_model(
    model_name=SVC_MODEL_NAME,
    model_version=SVC_MODEL_VERSION,
    model=clf,
    tags={"stage": "testing", "classifier_type": "svm.SVC", "svc_gamma": svc_gamma, "svc_C": svc_C},
    sample_input_data=test_features[:10],
)

### Deploy Model and Batch Inference

We can also deploy the model we saved to the registry to warehouse and predict it in the warehouse.

Although the model may contain multiple methods, every deployment can only have one target method, and you need to specify that when you deploy the model.

In [ ]:
svc_model.deploy(
    deployment_name="svc_model_predict",
    target_method="predict",
)

In [ ]:
remote_prediction = svc_model.predict(deployment_name="svc_model_predict", data=test_features)

print("Remote prediction:", remote_prediction[:10])

print("Result comparison:", np.array_equal(prediction, remote_prediction["output_feature_0"].values))

We can also deploy another method to warehouse.

In [ ]:
svc_model.deploy(
    deployment_name="svc_model_predict_proba",
    target_method="predict_proba",
)

In [ ]:
remote_prediction_proba = svc_model.predict(deployment_name="svc_model_predict_proba", data=test_features)

print("Remote prediction:", remote_prediction_proba[:10])

print("Result comparison:", np.allclose(prediction_proba, remote_prediction_proba.values))

## Use with customize model

### Download a GPT-2 model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

### Store GPT-2 Model components locally

In [ ]:
ARTIFACTS_DIR = "/tmp/gpt-2/"

In [ ]:
import os

os.makedirs(os.path.join(ARTIFACTS_DIR, "model"), exist_ok=True)
os.makedirs(os.path.join(ARTIFACTS_DIR, "tokenizer"), exist_ok=True)

model.save_pretrained(os.path.join(ARTIFACTS_DIR, "model"))
tokenizer.save_pretrained(os.path.join(ARTIFACTS_DIR, "tokenizer"))

### Create a custom model using GPT-2

In [ ]:
from snowflake.ml.model import custom_model
import pandas as pd


class GPT2Model(custom_model.CustomModel):
    def __init__(self, context: custom_model.ModelContext) -> None:
        super().__init__(context)

        self.model = AutoModelForCausalLM.from_pretrained(self.context.path("model"))
        self.tokenizer = AutoTokenizer.from_pretrained(self.context.path("tokenizer"))

    @custom_model.inference_api
    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        def _generate(input_text: str) -> str:
            input_ids = self.tokenizer.encode(input_text, return_tensors="pt")

            output = self.model.generate(input_ids, max_length=50, do_sample=True, top_p=0.95, top_k=60)
            generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

            return generated_text

        res_df = pd.DataFrame({"output": pd.Series.apply(X["input"], _generate)})
        return res_df

In [ ]:
gpt_model = GPT2Model(
    custom_model.ModelContext(
        models={},
        artifacts={
            "model": os.path.join(ARTIFACTS_DIR, "model"),
            "tokenizer": os.path.join(ARTIFACTS_DIR, "tokenizer"),
        },
    )
)

gpt_model.predict(pd.DataFrame({"input": ["Hello, are you GPT?"]}))

### Register the custom model

Here, how to specify dependencies and model signature manually is shown.

In [ ]:
GPT2_MODEL_NAME = "GPT2_MODEL"
GPT2_MODEL_VERSION = "v1"

In [ ]:
from snowflake.ml.model import model_signature

gpt_model_ref = registry.log_model(
    model_name=GPT2_MODEL_NAME,
    model_version=GPT2_MODEL_VERSION,
    model=gpt_model,
    conda_dependencies=["tensorflow", "transformers"],
    signatures={
        "predict": model_signature.ModelSignature(
            inputs=[model_signature.FeatureSpec(name="input", dtype=model_signature.DataType.STRING)],
            outputs=[model_signature.FeatureSpec(name="output", dtype=model_signature.DataType.STRING)],
        )
    },
)

### Deploy the model and predict

Relax version is an option that allow the deployer tries to relax the version specifications when initial attempt to
resolve the dependencies in Snowflake Anaconda Channel fails.

In [ ]:
gpt_model_ref.deploy(
    deployment_name="gpt_model_predict",
)

In [ ]:
gpt_model_ref.predict(deployment_name="gpt_model_predict", data=pd.DataFrame({"input": ["Hello, are you GPT?"]}))

## Use with XGBoost Model, Snowpark DataFrame and permanent deployment

### Prepare dataset

In [ ]:
from sklearn.datasets import fetch_kddcup99

DATA_TABLE_NAME = "KDDCUP99_DATASET"

kddcup99_data = fetch_kddcup99(as_frame=True)
kddcup99_sp_df = session.create_dataframe(kddcup99_data.frame)
kddcup99_sp_df.write.mode("overwrite").save_as_table(DATA_TABLE_NAME)

### Preprocessing Dataset

In [ ]:
from snowflake.ml.modeling.preprocessing import one_hot_encoder, ordinal_encoder, standard_scaler
import snowflake.snowpark.functions as F

quote_fn = lambda x: f'"{x}"'

ONE_HOT_ENCODE_COL_NAMES = ["protocol_type", "service", "flag"]
ORDINAL_ENCODE_COL_NAMES = ["labels"]
STANDARD_SCALER_COL_NAMES = [
    "duration",
    "src_bytes",
    "dst_bytes",
    "wrong_fragment",
    "urgent",
    "hot",
    "num_failed_logins",
    "num_compromised",
    "num_root",
    "num_file_creations",
    "num_shells",
    "num_access_files",
    "num_outbound_cmds",
    "count",
    "srv_count",
    "dst_host_count",
    "dst_host_srv_count",
]

TRAIN_SIZE_K = 0.2
kddcup99_data = session.table(DATA_TABLE_NAME)
kddcup99_data = kddcup99_data.with_columns(
    list(map(quote_fn, ONE_HOT_ENCODE_COL_NAMES + ORDINAL_ENCODE_COL_NAMES)),
    [
        F.to_char(col_name, "utf-8")
        for col_name in list(map(quote_fn, ONE_HOT_ENCODE_COL_NAMES + ORDINAL_ENCODE_COL_NAMES))
    ],
)
kddcup99_sp_df_train, kddcup99_sp_df_test = tuple(
    kddcup99_data.random_split([TRAIN_SIZE_K, 1 - TRAIN_SIZE_K], seed=2568)
)

ft_one_hot_encoder = one_hot_encoder.OneHotEncoder(
    handle_unknown="ignore",
    input_cols=list(map(quote_fn, ONE_HOT_ENCODE_COL_NAMES)),
    output_cols=ONE_HOT_ENCODE_COL_NAMES,
    drop_input_cols=True,
)
ft_one_hot_encoder = ft_one_hot_encoder.fit(kddcup99_sp_df_train)
kddcup99_sp_df_train = ft_one_hot_encoder.transform(kddcup99_sp_df_train)
kddcup99_sp_df_test = ft_one_hot_encoder.transform(kddcup99_sp_df_test)

ft_ordinal_encoder = ordinal_encoder.OrdinalEncoder(
    input_cols=list(map(quote_fn, ORDINAL_ENCODE_COL_NAMES)),
    output_cols=list(map(quote_fn, ORDINAL_ENCODE_COL_NAMES)),
    drop_input_cols=True,
)
ft_ordinal_encoder = ft_ordinal_encoder.fit(kddcup99_sp_df_train)
kddcup99_sp_df_train = ft_ordinal_encoder.transform(kddcup99_sp_df_train)
kddcup99_sp_df_test = ft_ordinal_encoder.transform(kddcup99_sp_df_test)

ft_standard_scaler = standard_scaler.StandardScaler(
    input_cols=list(map(quote_fn, STANDARD_SCALER_COL_NAMES)),
    output_cols=list(map(quote_fn, STANDARD_SCALER_COL_NAMES)),
    drop_input_cols=True,
)
ft_standard_scaler = ft_standard_scaler.fit(kddcup99_sp_df_train)
kddcup99_sp_df_train = ft_standard_scaler.transform(kddcup99_sp_df_train)
kddcup99_sp_df_test = ft_standard_scaler.transform(kddcup99_sp_df_test)

### Train an XGBoost model

In [ ]:
XGB_MODEL_NAME = "XGB_MODEL_KDDCUP99"
XGB_MODEL_VERSION = "v1"

In [ ]:
import xgboost

regressor = xgboost.XGBClassifier(objective="multi:softprob", n_estimators=500, reg_lambda=1, gamma=0, max_depth=5)
kddcup99_pd_df_train = kddcup99_sp_df_train.to_pandas()
regressor.fit(
    kddcup99_pd_df_train.drop(columns=["labels"]),
    kddcup99_pd_df_train["labels"],
)

### Log the model

In [ ]:
xgb_model = registry.log_model(
    model_name=XGB_MODEL_NAME,
    model_version=XGB_MODEL_VERSION,
    model=regressor,
    sample_input_data=kddcup99_sp_df_train.drop('"labels"'),
)

### Deploy the model permanently

In [ ]:
xgb_model.deploy(
    deployment_name="xgb_model_predict", target_method="predict", permanent=True, options={"relax_version": True}
)

### Predict with Snowpark DataFrame

In [ ]:
sp_res = xgb_model.predict(deployment_name="xgb_model_predict", data=kddcup99_sp_df_test)
sp_res.show()

### Prepare another SQL connection and registry

In [ ]:
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

another_session = Session.builder.configs(SnowflakeLoginOptions()).create()

### Call the deployed permanent UDF

In [ ]:
another_registry = model_registry.ModelRegistry(
    session=another_session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)
xgb_model_ref = model_registry.ModelReference(
    registry=another_registry,
    model_name=XGB_MODEL_NAME,
    model_version=XGB_MODEL_VERSION,
)
xgb_model_ref.list_deployments().show()

In [ ]:
sp_res = xgb_model_ref.predict(
    deployment_name="xgb_model_predict", data=another_session.create_dataframe(kddcup99_sp_df_test.to_pandas())
)
sp_res.show()

### Remove the deployed UDF

This would be done by calling delete_deployment in the registry.

In [ ]:
xgb_model_ref.delete_deployment(deployment_name="xgb_model_predict")

### Deploy to SPCS and using GPU for inference

In [ ]:
from snowflake.ml.model import deploy_platforms

xgb_model.deploy(
    deployment_name="xgb_model_predict_spcs",
    target_method="predict",
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options={"compute_pool": "...", "num_gpus": 1, "num_workers": 24},
)

In [ ]:
sp_res = xgb_model.predict(deployment_name="xgb_model_predict_spcs", data=kddcup99_sp_df_test)
sp_res.show()

In [ ]:
xgb_model.delete_deployment(deployment_name="xgb_model_predict_spcs")

## Using LLM with HuggingFace Pipeline

### Preparing Data into Snowflake

In [18]:
import pandas as pd
news_dataset = pd.read_json("News_Category_Dataset_v3.json", lines=True).convert_dtypes()

In [19]:
NEWS_DATA_TABLE_NAME = "news_dataset"
news_dataset_sp_df = session.create_dataframe(news_dataset)
news_dataset_sp_df.write.mode("overwrite").save_as_table(NEWS_DATA_TABLE_NAME)

In [35]:
news_dataset_sp = session.table(NEWS_DATA_TABLE_NAME).select('"headline"','"category"','"short_description"')

news_dataset_sp.show(max_width=600)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"headline"                                                                           |"category"  |"short_description"                                                                                                                                                                                                                                     |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
LLM_MODEL_NAME = "llama-2-7b-chat"
LLM_MODEL_VERSION = "v1"

In [22]:
from snowflake.ml.model.models import huggingface_pipeline

llama_model = huggingface_pipeline.HuggingFacePipelineModel(
    task="text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    token="...", # Put your HuggingFace token here.
    return_full_text=False,
    max_new_tokens=100,
    batch_size=1,
)

In [23]:
llama_model_ref = registry.log_model(
    model_name=LLM_MODEL_NAME,
    model_version=LLM_MODEL_VERSION,
    model=llama_model,
)

In [24]:
DEPLOYMENT_NAME="llama_predict"

In [ ]:
from snowflake.ml.model import deploy_platforms

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME,
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options={
        "compute_pool": "...",
        "num_gpus": 1,
    },
)

In [43]:
import snowflake.snowpark.functions as F

prompt_prefix = """[INST] <<SYS>>
Your output will be parsed by a computer program as a JSON object. Please respond ONLY with valid json that conforms to this JSON schema: {"properties": {"category": {"type": "string","description": "The category that the news should belong to."},"keywords": {"type": "array":"description": "The keywords that are mentioned in the news.","items": [{"type": "string"}]},"importance": {"type": "number","description": "A integer from 1 to 10 to show if the new is important. The higher the more important the news is."}},"required": ["properties","keywords","importance"]} 
As an example, input "Residents ordered to evacuate amid threat of growing wildfire in Washington state, medical facilities sheltering in place" results in the json: {"category": "Natural Disasters","keywords": ["evacuate", "wildfire", "Washington state", "medical facilities"],"importance": 8}
<</SYS>>
"""
prompt_suffix = "[/INST]"

input_df = news_dataset_sp.with_column(
    '"inputs"',
    F.concat_ws(
        F.lit(" "), F.lit(prompt_prefix), F.col('"headline"'), F.col('"short_description"'), F.lit(prompt_suffix)
    ),
)

input_df.show(max_width=600)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"headline"                                                                 

In [ ]:
res = llama_model_ref.predict(
    deployment_name=DEPLOYMENT_NAME,
    data=input_df
)

In [48]:
json_capture_regexp = r'[{\[]{1}([,:{}\[\]0-9.\-+Eaeflnr-u \n\r\t]|".*?")+[}\]]{1}'

output_json_col = F.parse_json(
    F.regexp_extract(
        F.replace(F.get(F.get(F.parse_json(F.col('"outputs"')), 0), F.lit("generated_text")), r"\"", '"'),
        json_capture_regexp,
        0,
    )
)

output_df = res.with_columns(
    ["pred_category", "pred_keywords", "pred_importance"],
    [
        F.get(output_json_col, F.lit("category")),
        F.get(output_json_col, F.lit("keywords")),
        F.get(output_json_col, F.lit("importance")),
    ],
)

output_df.show(max_width=600)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [49]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)

In [50]:
llama_model_ref.delete_model()